# 1.
Изучите данные из файлов с заказами и посетителями.
Столбцы таблицы с заказами (orders):
orderId — идентификатор заказа;
userId — идентификатор пользователя, совершившего заказ;
group — группа A/B-теста;
revenue — выручка заказа;
date — дата заказа.
Столбцы таблицы с посетителями (visitors):
date — дата;
group — группа A/B-теста;
visitors — количество посетителей интернет-магазина в указанной группе A/B-теста в указанную дату.
Не удаляйте вывод первых пяти строк датафреймов и выполните задание по шагам:
1) Создайте датафрейм datesGroups с уникальными парами значений 'date' и 'group', таблицы orders. Избавьтесь от дубликатов методом drop_duplicates().
2) Объявите переменную ordersAggregated, содержащую:
дату;
группу A/B-теста;
число уникальных заказов в группе теста по указанную дату включительно;
число уникальных пользователей, совершивших хотя бы 1 заказ в группе теста по указанную дату включительно;
суммарную выручку заказов в группе теста по указанную дату включительно.
3) Объявите переменную visitorsAggregated, содержащую:
дату;
группу A/B-теста;
количество уникальных посетителей в группе теста по указанную дату включительно.
4) ordersAggregated и visitorsAggregated отсортируйте по столбцам 'date', 'group' в указанном порядке.
5) Определите переменную cumulativeData, объединив ordersAggregated и visitorsAggregated по колонкам 'date', 'group' методом merge()
6) Присвойте столбцам cumulativeData названия ['date', 'group', 'orders', 'buyers', 'revenue', 'visitors']
7) Выведите первые 5 строк переменной cumulativeData

Подсказка
Получите таблицу ordersAggregated так:
datesGroups.apply(
    lambda x: orders[np.logical_and(orders['date'] <= x['date'], orders['group'] == x['group'])].agg({
'date' : 'max',
'group' : 'max',
'orderId' : pd.Series.nunique,
'userId' : pd.Series.nunique,
'revenue' : 'sum'}), axis=1).sort_values(by=['date','group'])
Объедините таблицы методом merge():
cumulativeData = ordersAggregated.merge(visitorsAggregated, left_on=['date', 'group'], right_on=['date', 'group'])

In [20]:
import pandas as pd
import datetime as dt
import numpy as np

orders = pd.read_csv('datasets/data_for_tasks_3.csv', sep=',')
orders['date'] = orders['date'].map(
    lambda x: dt.datetime.strptime(x, '%d/%m/%Y')
)

visitors = pd.read_csv('datasets/data_for_tasks_3_visitors.csv', sep=',')
visitors['date'] = visitors['date'].map(
    lambda x: dt.datetime.strptime(x, '%d/%m/%Y')
)

print(orders.head(5))
print(visitors.head(5))

# Создайте датафрейм datesGroups с уникальными парами значений 'date' и 'group', таблицы orders. Избавьтесь от дубликатов методом drop_duplicates()
datesGroups = orders[['date', 'group']].drop_duplicates()

# Объявите переменную ordersAggregated, содержащую:
# дату;
# группу A/B-теста;
# число уникальных заказов в группе теста по указанную дату включительно;
# число уникальных пользователей, совершивших хотя бы 1 заказ в группе теста по указанную дату включительно;
# суммарную выручку заказов в группе теста по указанную дату включительно.
ordersAggregated = datesGroups.apply(lambda x: orders[np.logical_and(orders['date'] <= x['date'],
                                                                     orders['group'] == x['group'])]\
                                     .agg({'date':np.max,
                                           'group':np.max,
                                           'orderId':'nunique',
                                           'userId':'nunique',
                                           'revenue':np.sum}), axis='columns')\
                              .sort_values(by=list(datesGroups.columns))

# Объявите переменную visitorsAggregated, содержащую:
# дату;
# группу A/B-теста;
# количество уникальных посетителей в группе теста по указанную дату включительно
visitorsAggregated = datesGroups.apply(lambda x: visitors[np.logical_and(visitors['date'] <= x['date'],
                                                                         visitors['group'] == x['group'])]
                                       .agg({'date': np.max,
                                             'group':np.max,
                                             'visitors':np.sum}), axis='columns')\
                                       .sort_values(by=list(datesGroups.columns))

# Определите переменную cumulativeData, объединив ordersAggregated и visitorsAggregated по колонкам 'date', 'group' методом merge()
cumulativeData = ordersAggregated.merge(right=visitorsAggregated, left_on=['date', 'group'], right_on=['date', 'group'])

# Присвойте столбцам cumulativeData названия ['date', 'group', 'orders', 'buyers', 'revenue', 'visitors']
cumulativeData.columns = ['date', 'group', 'orders', 'buyers', 'revenue', 'visitors']

print(cumulativeData.head())

         userId group       orderId  revenue       date
0  1.815263e+19     B  4.612878e+15      489 2019-04-22
1  1.815263e+19     B  4.612878e+15      489 2019-04-22
2  1.794078e+19     B  4.136278e+18       97 2019-04-22
3  1.794078e+19     B  4.136278e+18      279 2019-04-22
4  2.461477e+18     B  1.406554e+19     4092 2019-04-22
        date group  visitors
0 2019-03-11     A       321
1 2019-03-12     A       831
2 2019-03-13     A       700
3 2019-03-14     A      1222
4 2019-03-15     A       859
        date group  orders  buyers  revenue  visitors
0 2019-03-11     A      10      10   110291       321
1 2019-03-11     B       9       9    36646       337
2 2019-03-12     A      35      33   343089      1152
3 2019-03-12     B      35      33    90491      1198
4 2019-03-13     A      59      56  1012663      1852
